In [1]:
from bggcomplex import *
import numpy as np

In [2]:
from multiprocessing import Pool

In [25]:
BGG = BGGComplex("B4")
BGG.find_cycles();

In [30]:
from sage.parallel.multiprocessing_sage import parallel_iter

In [31]:
Solver = BGGMapSolver(BGG,BGG.zero_root)

In [32]:
Solver.solve();

KeyError: 'PBW[-alpha[2]]*PBW[-alpha[1] - alpha[2]]*PBW[-alpha[2] - alpha[3] - 2*alpha[4]]*PBW[-alpha[1] - alpha[2] - alpha[3] - 2*alpha[4]]'

In [ ]:
print(Solver.timer)

In [ ]:
Solver.check_maps()

In [ ]:
BGG.compute_signs()

In [ ]:
BGG.plot_graph().plot()

In [20]:
@parallel(3)
def f(l):
    for n in l:
        n+n
    return None

for _ in f([[1]*1000]*3):
    pass